In [33]:
# Initial Imports
import pandas as pd
from pathlib import Path
import hvplot.pandas
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Netflix Subscriber Data

In [ ]:
# Read NFLX_sub.csv into Pandas DataFrame
subs_df = pd.read_csv(Path('Resources/NFLX_sub.csv'))

# View first five rows of subs_df
subs_df.head()

,Area,Years,Subscribers
0,United States and Canada,Q1 - 2018,60909000
1,"Europe, Middle East and Africa",Q1 - 2018,29339000
2,Latin America,Q1 - 2018,21260000
3,Asia-Pacific,Q1 - 2018,7394000
4,United States and Canada,Q2 - 2018,61870000


In [35]:
# Drop 'Area' column and rename 'Years' to 'Date'
subs_df = subs_df.drop(columns = 'Area')
subs_df = subs_df.rename(columns = {'Years' : 'Date'})
subs_df.head()

,Date,Subscribers
0,Q1 - 2018,60909000
1,Q1 - 2018,29339000
2,Q1 - 2018,21260000
3,Q1 - 2018,7394000
4,Q2 - 2018,61870000


In [36]:
# Check datatypes
subs_df.dtypes

Date           object
Subscribers     int64
dtype: object

In [37]:
# Convert 'Date' into datetime object
subs_df['Date'] = pd.to_datetime(
    subs_df['Date'].str.replace(r'(Q\d) - (\d+)', r'\2-\1'))

subs_df.dtypes

Date           datetime64[ns]
Subscribers             int64
dtype: object

In [38]:
# Set 'Date' as index
subs_df = subs_df.set_index('Date')

In [48]:
# Find the sum of all subscribers by year using the groupby function
yearly_subs = subs_df.groupby(by = [subs_df.index.year]).sum()

#Plot results
yearly_subs.hvplot(yformatter = '%.0f', title = 'Total Netflix Subscribers By Year')

:Curve   [Date]   (Subscribers)

## Stop to Think!
What sort of trend did we see from 2018 - 2020? What are some possible reasons for growth and decline? Knowing what we do about 2020, is this graph an accurate representation if we were to forcast future subscriber growth?

# Netflix Closing Price

In [43]:
# Read NFLX_stock.csv into Pandas DataFrame and set the index column as 'Date'
close_df = pd.read_csv(Path('Resources/NFLX_stock.csv'), 
                       index_col = 'Date', 
                       infer_datetime_format = True, 
                       parse_dates = True)

# Display the first five rows of close_df
close_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-06-06,164.949997,166.820007,164.509995,165.169998,165.169998,4382100
2017-06-07,165.600006,166.399994,164.410004,165.610001,165.610001,3353100
2017-06-08,166.119995,166.869995,164.839996,165.880005,165.880005,3719100
2017-06-09,166.270004,166.270004,154.500000,158.029999,158.029999,10292000
2017-06-12,155.300003,155.529999,148.309998,151.440002,151.440002,14114500


In [44]:
# Use loc function to only read 'Close' time series data
close_df = close_df.loc[:, 'Close']

close_df.head()

,Close
Date,
2017-06-06,165.169998
2017-06-07,165.610001
2017-06-08,165.880005
2017-06-09,158.029999
2017-06-12,151.440002


In [ ]:
# Convert from Series to DataFrame
close_df = close_df.to_frame()

# Display top 5 rows
close_df.head()

In [45]:
# Use hvplot and the loc function to show closing prices in 2019
close_df.loc['2019'].hvplot(title = 'NFLX Closing Prices: 2019')

:Curve   [Date]   (Close)

## Stop to think!
If subscriber count was growing in 2019, but dropped in 2020, is the trend we are seeing in our closing price data expected? DisneyPlus became available on November 2019, could that have affected stock price?

In [47]:
# Concat close_df and subs_df
nflx_df = pd.concat([close_df, subs_df])

# Find the average closing price and number of subscribers group by year and quarter
qtrly_nflx = nflx_df.groupby(by = [nflx_df.index.year, nflx_df.index.quarter]).mean().dropna()

# View first five rows of qtrly_nflx
qtrly_nflx.head()

Close  Subscribers
Date Date                         
2018 1     272.217869   29725500.0
     2     340.896877   31088500.0
     3     362.951427   32605500.0
     4     299.257778   34814750.0
2019 1     346.668853   37215750.0

In [ ]:
# Flatten multiIndex and rename 'Date' column as 'Quarter
qtrly_nflx.reset_index(level = 1, inplace = True)
qtrly_nflx = qtrly_nflx.rename(columns = {'Date' : 'Quarter'})

# Display last five rows of qtrly_nflx
qtrly_nflx.tail()

In [14]:
# Create column for Daily Returns and use the pct_change function to calculate
qtrly_nflx['Daily Returns'] = qtrly_nflx['Close'].pct_change()

In [15]:
# Using Pandas corr function, look for any correlations between average closing prices and number of subscribers
qtrly_nflx[['Subscribers', 'Daily Returns']].corr()

,Subscribers,Daily Returns
Subscribers,1.000000,0.151713
Daily Returns,0.151713,1.000000
